# Reconocimiento de patrones: Identificación de grupos o Clustering
### Ramón Soto C. [(rsotoc@moviquest.com)](mailto:rsotoc@moviquest.com/)
![ ](images/blank.png)
![agents](images/binary_data_under_a_magnifying.jpg)
[ver en nbviewer](http://nbviewer.ipython.org/github/rsotoc/pattern-recognition/blob/master/Clustering%20IV.ipynb)

## Técnicas de clustering: *ISODATA*

La técnica de *k-medias* (o *k-means*) es una de las técnicas de clustering más simples y más utilizadas; es una técnica muy simple y rápida. Sin embargo, es fuertemente dependiente de la selección de $k$ y este parámetro no puede modificarse una vez que inicia el proceso. Existen diversos métodos relacionado con $k$-medias que buscan resolver alguna de sus limitantes, entre los que se encuentran $k$-medianas, $k$-medoides, $k$-medias difuso.

El método **ISODATA** (*Iterative Self-Organizing Data Analysis Techniques*) es un algoritmo similar al $k$-means; su objetivo es particionar un conjunto de datos en subconjuntos. Sin embargo, a diferencia de $k$-means, el método Isodata maneja una serie de [*heurísticas*](https://en.wikipedia.org/wiki/Heuristic) con tres objetivos:

* Eliminar clusters con poco ejemplares.
* Unir clusters muy cercanos.
* Dividir clusters dispersos

![ ](images/isodata-1.png)


### Parámetros

El algoritmo Isodata utiliza los siguientes parámetros:

* $k_{init}$: Número inicial de clusters.
* $k$: el número actual de clusters
* $n_{min}$: Mínimo número de elementos en un cluster.
* $I_{max}$: Máximo número de iteraciones.
* $\sigma_{max}$: Máximo valor de la desviación estándar de los puntos al centro de su cluster, a lo largo de cada eje.
* $L_{min}$: Distancia mínima entre los centroides de las clases.
* $P_{max}$: Número máximo de clusters que pueden ser unificados en una iteración.


### Algoritmo

El algoritmo Isodata sigue los siguientes pasos (dado un conjunto de datos):

In [1]:
# Inicializar el ambiente
import numpy as np
import pandas as pd
import math
import random
import time
import os
import sys
from scipy.spatial import distance
np.set_printoptions(precision=2, suppress=True) # Cortar la impresión de decimales a 1

os.chdir('Data sets')
LARGER_DISTANCE = sys.maxsize
TALK = True # TALK = True, imprime resultados parciales

In [2]:
# Leer los datos de archivo
df = pd.read_csv("datosProm.csv", names = ['A', 'B'])
print (df.describe())

DATA_SET = df.values
DATA_LEN = len(DATA_SET)

# Definir una clase para expresar puntos y su asignación a un cluster
class DataPoint:
    def __init__(self, p):
        self.value = p[:]
        
    def set_value(self, p):
        self.value = p
    
    def get_value(self):
        return self.value
    
    def set_cluster(self, cluster):
        self.cluster = cluster
    
    def get_cluster(self):
        return self.cluster

data = []
def initialize_dataset():
    for i in range(DATA_LEN):
        point = DataPoint(DATA_SET[i])
        point.set_cluster(None)
        data.append(point)
    return

# --------------------------
# Crear el conjunto de datos
initialize_dataset()

                A          B
count   30.000000  30.000000
mean    62.576333  62.595633
std     35.940813  29.057323
min      0.000000   0.000000
25%     68.720000  57.262500
50%     77.345000  75.655000
75%     86.000000  83.262750
max    100.000000  95.063000


1) Definir los valores de $k_{init}, n_{min}, I_{max}, \sigma_{max}, L_{min}$ y $P_{max}$:

In [3]:
K_INIT = 5
N_MIN = 3
I_MAX = 10
S_MAX = 5
L_MIN = 80
P_MAX = 2

num_clusters = 5 # valor de k
iteration = 0

2) Seleccionar arbitrariamente los centroides iniciales 

In [4]:
# Definir forma de muestreo; 0 = random, 1=head, 2=tail
SAMPLING_METHOD = 1 

centroids = []
def initialize_centroids():
    if (TALK) : 
        print("Centroides inicializados en:")
    for c in range(num_clusters):
        if (SAMPLING_METHOD == 0) :
            which = random.randint(0,DATA_LEN-1)
        elif (SAMPLING_METHOD == 1):
            which = c
        else :
            which = DATA_LEN-1 - c
                
        centroids.append(list(DATA_SET[which]))
        if (TALK) : 
            print(centroids[c])        
    if (TALK) : 
        print()
    
    return

# --------------------------
# Inicializar los centroides
initialize_centroids()

Centroides inicializados en:
[70.280000000000001, 42.125]
[0.0, 56.75]
[79.0, 2.5]
[75.640000000000001, 11.667]
[82.0, 58.799999999999997]



3) Asignar cada punto del conjunto de datos al cluster donde la distancia del punto al centroide es menor. 

4) Eliminar los clusters con menos de $n_{min}$ elementos. Ajustar el valor de $k$ y reetiquetar los clusters.

In [5]:
elim = 0
members = []
def update_clusters():
    global num_clusters, elim, members
    changed = False
    
    if (TALK) :
        print("Actualizando clusters")
    for i in range(DATA_LEN):
        minDistance = LARGER_DISTANCE
        currentCluster = 0
        
        for j in range(num_clusters):
            dist = distance.euclidean(data[i].get_value(), centroids[j])
            if(dist < minDistance):
                minDistance = dist
                currentCluster = j
        
        if(data[i].get_cluster() is None or data[i].get_cluster() != currentCluster):
            data[i].set_cluster(currentCluster)
            changed = True
            
    members = [0] * num_clusters
    for i in range(DATA_LEN):
        members[data[i].get_cluster()] += 1
    
    if (TALK) : 
        for j in range(num_clusters):
            print("El cluster ", j, " incluye ", members[j], "miembros.")
        print()
        
    elim = 0
    for j in range(num_clusters):
        if (members[j] < N_MIN):
            if (TALK) :
                print("Eliminando cluster ", j)
            for i in range(DATA_LEN):
                cluster = data[i].get_cluster()
                if (cluster == j-elim) :
                    data[i].set_cluster(None)
                elif (cluster != None and cluster > j-elim) :
                    data[i].set_cluster(cluster-1)
            elim += 1
            members[j] = 0
    
    if (TALK and elim > 0) : 
        for j in range(num_clusters):
            print("El cluster ", j, " incluye ", members[j], "miembros.")
        print()
    num_clusters -= elim

    return changed

# --------------------------
# Actualizar los clusters
KEEP_WALKING = update_clusters()

Actualizando clusters
El cluster  0  incluye  2 miembros.
El cluster  1  incluye  7 miembros.
El cluster  2  incluye  3 miembros.
El cluster  3  incluye  2 miembros.
El cluster  4  incluye  16 miembros.

Eliminando cluster  0
Eliminando cluster  3
El cluster  0  incluye  0 miembros.
El cluster  1  incluye  7 miembros.
El cluster  2  incluye  3 miembros.
El cluster  3  incluye  0 miembros.
El cluster  4  incluye  16 miembros.



5) Recalcular los centroides a partir de los puntos actualmente en cada cluster. Si se eliminaron clusters en el paso 4) el algoritmo regresa la paso 3).

In [6]:
def update_centroids():
    global centroids
    centroids = []

    if (TALK) : 
        print("Los nuevos centroids son:")
    for j in range(num_clusters):
        means = [0] * DATA_SET.shape[1]
            
        clusterSize = 0
        for k in range(len(data)):
            if(data[k].get_cluster() == j):
                p = data[k].get_value()
                for i in range(DATA_SET.shape[1]):
                    means[i] += p[i]
                clusterSize += 1

        if(clusterSize > 0):
            for i in range(DATA_SET.shape[1]):
                means[i] = means[i] / clusterSize
            centroids.append(means)

        if (TALK) : 
            print(centroids[j])        
    if (TALK) : 
        print()
    
    return

# --------------------------
# Actualizar los centroides
update_centroids()
if (elim > 0) :
    KEEP_WALKING = update_clusters()
    update_centroids()

Los nuevos centroids son:
[0.0, 73.886571428571415]
[88.90666666666668, 2.5]
[82.259999999999991, 78.200249999999983]

Actualizando clusters
El cluster  0  incluye  7 miembros.
El cluster  1  incluye  6 miembros.
El cluster  2  incluye  17 miembros.

Los nuevos centroids son:
[0.0, 73.886571428571415]
[81.808333333333323, 11.222333333333333]
[81.555294117647065, 76.078176470588218]



6) Calcular las distancias promedio $\Delta_j$ de los puntos de un cluster a su centroide y la distancia promedio general $\Delta$.

In [7]:
deltas = []
delta = 0
def update_deltas():
    global deltas, delta
    deltas = [0] * num_clusters
    delta = 0
    
    for i in range(DATA_LEN):
        cluster = data[i].get_cluster()
        deltas[cluster] += distance.euclidean(data[i].get_value(), centroids[cluster])
    mem = 0
    for i in range(num_clusters):
        delta += deltas[i]
        mem += members[i]
        deltas[i] /= members[i]
        if (TALK) : 
            print("Distancia promedio en el cluster {}:".format(i), deltas[i])        
    delta /= mem
    if (TALK) : 
        print("Distancia promedio global: {}\n".format(delta))
    
    return
    
update_deltas()

Distancia promedio en el cluster 0: 8.882897959183678
Distancia promedio en el cluster 1: 12.762142305287727
Distancia promedio en el cluster 2: 13.744777384565223
Distancia promedio global: 12.413811836120695



7) Si esta es la última iteración, terminar. En caso contrario verificar si quedan la mitad o menos de los clusters iniciales y de ser así ir al paso 8 (dividir clusters). En caso contrario, si la iteración es par o el número de clusters es mayor que el doble de los clusters iniciales, entonces ir al paso 9 (unir). En caso contrario, volver al paso 3 (como $k$-means).

In [8]:
# Ejecutar sólo desués de haber "activado" los pasos 8 y 9
#while(iteration < I_MAX and KEEP_WALKING) :
#    if (num_clusters <= K_INIT / 2) :
#        divide_clusters()
#    elif (iteration % 2 == 0 or num_clusters > 2 * K_INIT) :
#        mix_clusters()
#        
#    KEEP_WALKING = update_clusters()
#    if (KEEP_WALKING):
#        update_centroids()
#    else :
#        if (TALK) : 
#            print ("No más cambios.")
#    iteration += 1

8) Calcular, para cada cluster $S_j$ un vector $\mathbf{s_j} = (s_{j1}\ldots s_{jn_j})$ con las desviaciones estándar $s_{ji}$ de cada atributo $i$ de los elementos $\mathbf{x} = (x_{1}\ldots x_{n_j})$ en el cluster (con respecto al centroide $\mathbf{z}_j$).

$$s_{ji}=\frac{1}{n_j}\sqrt{\sum_{\mathbf{x}\in S} (x_i-z_{ji})^2 }$$

Si al menos una componente del vector de desviaciones estándar sobrepasa la máxima desviación estándar permitida, $\sigma_{max}$ consideramos dividir la clase, siempre que se cumpla alguna de las siguientes condiciones adicionales:

* Que la distancia promedio entre puntos en el cluster sea mayor que la distancia promedio global y que el cluster tenga más del doble de elementos que el mínimo permitido para un cluster ($n_{min}$), o

* Que el número actual de clusters sea menor que la mitad del número de clusters inicial.

Si se cumple alguna de estas condiciones, reemplazar el cluster por dos nuevos clusters. Seleccionar como centroides de estos clusters los dos vectores más alejados entre sí en el cluster original.

Si hubo al menos una división de clusters, volver a reconstruir todos los clusters (y centroides).

In [9]:
def divide_clusters():
    global num_clusters
    # Cálculo de desviaciones estandar
    sigma_vect = [[0] * DATA_SET.shape[1]] * num_clusters
    for d in range(DATA_LEN):
        cluster = data[d].get_cluster()
        p = data[d].get_value()
        for i in range(DATA_SET.shape[1]):
            sigma_vect[cluster][i] += (p[i] - centroids[cluster][i])**2        
    candidates = []
    for cluster in range(num_clusters):
        for i in range(DATA_SET.shape[1]):
            sigma_vect[cluster][i] = math.sqrt(sigma_vect[cluster][i]) / members[cluster]
            if (sigma_vect[cluster][i] > S_MAX):
                candidates.append(cluster)
                break # Sucio... pero eficiente :-)
    
    divided = False
    for cluster in candidates:
        cond = num_clusters < K_INIT/2 or (deltas[cluster] > delta and members[cluster] > N_MIN)
        if(cond) :
            centroids.pop(cluster)
            points = []
            for d in range(DATA_LEN):
                if (data[d].get_cluster() == cluster):
                    points.append(data[d].get_value())
            dist = distance.squareform(distance.pdist(points, 'euclidean'))
            idx = (dist==dist.max()).argmax()
            z1 = list(points[idx // len(points)])
            z2 = list(points[idx % len(points)])
            if (TALK) :
                print("Se dividirá el cluster {}.\nSe crearán nuevos clusters en {} y {}.\n"
                     .format(cluster, z1, z2))
            centroids.append(z1)
            centroids.append(z2)
            num_clusters += 1
            divided = True

    if (divided) :
        update_clusters()
        update_centroids()
    
    return 

divide_clusters()

Se dividirá el cluster 1.
Se crearán nuevos clusters en [91.719999999999999, 0.0] y [70.799999999999997, 35.0].

Actualizando clusters
El cluster  0  incluye  7 miembros.
El cluster  1  incluye  16 miembros.
El cluster  2  incluye  5 miembros.
El cluster  3  incluye  2 miembros.

Eliminando cluster  3
El cluster  0  incluye  7 miembros.
El cluster  1  incluye  16 miembros.
El cluster  2  incluye  5 miembros.
El cluster  3  incluye  0 miembros.

Los nuevos centroids son:
[0.0, 73.886571428571415]
[82.259999999999991, 78.200249999999983]
[84.009999999999991, 6.466800000000001]



9) Calcular las distancias *inter-clusters* entre todos los centroides de los clusters. Seleccionar el máximo número permitido de pares de clusters para ser unificados ($P_{max}$), que satisfagan las siguientes condiciones:

* Que la distancia entre los centroides sea menor que la mínima distancia permitida ($L_{min}$), y

* Que ninguno de los dos clusters haya participado en una unificación en la presente iteración.

Si se cumplen estas condiciones, reemplazar los dos cluster por un nuevo cluster cuyo centroide es el punto medio de los centroides originales.

Si hubo al menos una unificación de clusters, volver a reconstruir todos los clusters.

In [10]:
def mix_clusters():
    global centroids, num_clusters
    dist = distance.squareform(distance.pdist(centroids, 'euclidean'))
    flag = math.floor(dist.max() * 10)
    dist[dist == 0] = flag
    
    mixed = False
    while (dist.min() < flag):
        idx = (dist==dist.min()).argmax()
        z1 = idx // len(centroids)
        z2 = idx % len(centroids)
        
        if (dist.min() < L_MIN):
            dist[z1] = flag
            dist[:,z1] = flag
            dist[z2] = flag
            dist[:,z2] = flag
            z = [sum(x)/2 for x in zip(centroids[z1], centroids[z2])]
            centroids[z1] = z
            centroids[z2] = [LARGER_DISTANCE]*DATA_SET.shape[1]
            num_clusters -= 1

            mixed = True
            if(TALK):
                print("Unificando clusters {} y {}\nSe creará nuevo centroide en {}\n"
                      .format(z1, z2, z))
        else :
            dist[z1][z2] = flag
            dist[z2][z1] = flag
        
    if (mixed) :
        update_clusters()
        update_centroids()

    return

mix_clusters()

Unificando clusters 1 y 2
Se creará nuevo centroide en [83.134999999999991, 42.333524999999995]

Actualizando clusters
El cluster  0  incluye  7 miembros.
El cluster  1  incluye  23 miembros.

Los nuevos centroids son:
[0.0, 73.886571428571415]
[81.621304347826069, 59.159260869565216]



In [11]:
# Reproducido aquí para facilitar la ejecución
iteration +=1
while(iteration < I_MAX and KEEP_WALKING) :
    if (num_clusters <= K_INIT / 2) :
        divide_clusters()
    elif (iteration % 2 == 0 or num_clusters > 2 * K_INIT) :
        mix_clusters()
        
    KEEP_WALKING = update_clusters()
    if (KEEP_WALKING):
        update_centroids()
    else :
        if (TALK) : 
            print ("No más cambios.")
    iteration += 1

Se dividirá el cluster 0.
Se crearán nuevos clusters en [0.0, 56.75] y [0.0, 90.625].

Se dividirá el cluster 1.
Se crearán nuevos clusters en [91.719999999999999, 0.0] y [100.0, 95.062999999999988].

Actualizando clusters
El cluster  0  incluye  12 miembros.
El cluster  1  incluye  7 miembros.
El cluster  2  incluye  5 miembros.
El cluster  3  incluye  6 miembros.

Los nuevos centroids son:
[79.186666666666667, 66.520416666666662]
[0.0, 73.886571428571415]
[84.009999999999991, 6.466800000000001]
[84.5, 88.347333333333324]

Actualizando clusters
El cluster  0  incluye  7 miembros.
El cluster  1  incluye  7 miembros.
El cluster  2  incluye  6 miembros.
El cluster  3  incluye  10 miembros.

Los nuevos centroids son:
[80.997142857142862, 62.769571428571432]
[0.0, 73.886571428571415]
[81.808333333333323, 11.222333333333333]
[81.945999999999998, 85.394199999999984]

Unificando clusters 0 y 3
Se creará nuevo centroide en [81.471571428571423, 74.081885714285704]

Actualizando clusters
El clus

El método ISODATA es un método poco utilizado fuera del área de procesamiento de imágenes. Es difícil de configurar, por la cantidad y naturaleza de los parámetros a definir.

<hr style="border-width: 3px;">

### Tarea 6

* Generar un programa integrado del método ISODATA 

* Aplicar los elementos vistos en clase hasta el momento a un conjunto de datos de su preferencia

**Fecha de entrega**: Martes 20 de septiembre.